In [1]:
import pandas as pd
import tsfresh
import os
import json
import scapy
import numpy as np
import warnings
from timeit import default_timer as timer

from scapy.all import *

warnings.filterwarnings("ignore") #ignore warnings caused by 

In [2]:
paths = [
    '../Data/network-traffic-dataset/output_bottom.csv',
    '../Data/network-traffic-dataset/output_left.csv',
    '../Data/network-traffic-dataset/output_right.csv'
]
df_w = pd.concat([pd.read_csv(p) for p in paths], ignore_index=True)

df1 = pd.read_csv('../Data/malicious/WebOS_binary.csv') #
df2 = pd.read_csv('../Data/malicious/Server_Binary.csv') #
df3 = pd.read_csv('../Data/malicious/Raspberry_Webmine_Robust.csv')
df4 = pd.read_csv('../Data/malicious/Raspberry_Binary.csv') #
df5 = pd.read_csv('../Data/malicious/Raspberry_Webmine_Aggressive.csv')
df6 = pd.read_csv('../Data/malicious/Raspberry_WebminePool_Aggressive.csv')
df7 = pd.read_csv('../Data/malicious/Server_WebminePool_Aggressive.csv') #
df32 = pd.read_csv('../Data/malicious/Server_WebminePool_Robust.csv') #
df33 = pd.read_csv('../Data/malicious/Raspberry_WebminePool_Stealthy.csv') #
df34 = pd.read_csv('../Data/malicious/Raspberry_WebminePool_Robust.csv') #
df35 = pd.read_csv('../Data/malicious/Desktop_WebminePool_Aggressive.csv') #

df_results = pd.DataFrame()

In [3]:
#################################################################
#                                                               #
#                     Filtering                                 #
#                                                               #
#################################################################

# For WebOS = 18:56:80:17:d0:ef
index_names = df1[((df1['HW_dst'] != '18:56:80:17:d0:ef') & (df1['Hw_src'] != '18:56:80:17:d0:ef'))].index
df1.drop(index_names, inplace = True)

# Big_Server_Monero_mining_data = a4:bb:6d:ac:e1:fd

index_names = df2[((df2['HW_dst'] != 'a4:bb:6d:ac:e1:fd') & (df2['Hw_src'] != 'a4:bb:6d:ac:e1:fd'))].index
df2.drop(index_names, inplace = True)

# ege_data_rasberry = dc:a6:32:67:66:4b	

index_names = df3[((df3['HW_dst'] != 'dc:a6:32:67:66:4b') & (df3['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df3.drop(index_names, inplace = True)

# Rasberry_binary_monero_mining = dc:a6:32:68:35:8a

index_names = df4[((df4['HW_dst'] != 'dc:a6:32:68:35:8a') & (df4['Hw_src'] != 'dc:a6:32:68:35:8a'))].index
df4.drop(index_names, inplace = True)

# Rasberry_network_data_2 = dc:a6:32:67:66:4b

index_names = df5[((df5['HW_dst'] != 'dc:a6:32:67:66:4b') & (df5['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df5.drop(index_names, inplace = True)

# Rasberry-Webmine = dc:a6:32:67:66:4b
index_names = df6[((df6['HW_dst'] != 'dc:a6:32:67:66:4b') & (df6['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df6.drop(index_names, inplace = True)

# Server_Webmine_Network_data = a4:bb:6d:ac:e1:fd

index_names = df7[((df7['HW_dst'] != 'a4:bb:6d:ac:e1:fd') & (df7['Hw_src'] != 'a4:bb:6d:ac:e1:fd'))].index
df7.drop(index_names, inplace = True)

# Server_%50_Mining = a4:bb:6d:ac:e1:fd

index_names = df32[((df32['HW_dst'] != 'a4:bb:6d:ac:e1:fd') & (df32['Hw_src'] != 'a4:bb:6d:ac:e1:fd'))].index
df32.drop(index_names, inplace = True)

# Rasberry_webmine_%10 = dc:a6:32:67:66:4b

index_names = df33[((df33['HW_dst'] != 'dc:a6:32:67:66:4b') & (df33['Hw_src'] != 'dc:a6:32:67:66:4b'))].index
df33.drop(index_names, inplace = True)

# Rasberry_webmine_%50 = dc:a6:32:68:35:8a

index_names = df34[((df34['HW_dst'] != 'dc:a6:32:68:35:8a') & (df34['Hw_src'] != 'dc:a6:32:68:35:8a'))].index
df34.drop(index_names, inplace = True)

# Desktop_Webmine_%100 = dc:a6:32:68:35:8a

index_names = df35[((df35['HW_dst'] != 'd8:3b:bf:8f:ba:ba') & (df35['Hw_src'] != 'd8:3b:bf:8f:ba:ba'))].index
df35.drop(index_names, inplace = True)

In [4]:
#################################################################
#                                                               #
#      Labeling Features for further calculations               #
#                                                               #
#################################################################

df1.insert(7, "Is_malicious", 1)
df2.insert(7, "Is_malicious", 1)
df3.insert(7, "Is_malicious", 1)
df4.insert(7, "Is_malicious", 1)
df5.insert(7, "Is_malicious", 1)
df6.insert(7, "Is_malicious", 1)
df7.insert(7, "Is_malicious", 1)
df32.insert(7, "Is_malicious", 1)
df33.insert(7, "Is_malicious", 1)
df34.insert(7, "Is_malicious", 1)
df35.insert(7, "Is_malicious", 1)

In [5]:
def flows_to_pseudopackets(df_flows, is_malicious: int):
    df = df_flows.copy()

    # ensure needed columns exist, then coerce to numeric
    need = ['start','end','sPackets','rPackets','sBytesAvg','rBytesAvg','sBytesSum','rBytesSum']
    for c in need:
        if c not in df.columns:
            df[c] = np.nan
    df[need] = df[need].apply(pd.to_numeric, errors='coerce')

    rows = []
    # using enumerate to get a stable integer flow_id
    for flow_id, (_, row) in enumerate(df.fillna(0).iterrows()):
        start = float(row['start']); end = float(row['end'])
        dur = max(end - start, 1e-6)

        sN = int(row['sPackets']); rN = int(row['rPackets'])

        # robust: prefer Avg if it's finite and > 0, else fallback to Sum/Packets
        def _pkt_len(avg, total, n):
            if pd.notna(avg) and avg > 0:
                return float(avg)
            return (float(total) / n) if (pd.notna(total) and n > 0) else 0.0

        s_len = _pkt_len(row['sBytesAvg'], row['sBytesSum'], sN)
        r_len = _pkt_len(row['rBytesAvg'], row['rBytesSum'], rN)

        N = sN + rN
        if N == 0:
            continue

        # distribute timestamps across the flow window
        for k in range(sN):
            t = start + dur * (k + 0.5) / N
            rows.append({'flow_id': flow_id, 'Time': t, 'Length': s_len, 'Is_malicious': is_malicious})
        for k in range(rN):
            t = start + dur * (sN + k + 0.5) / N
            rows.append({'flow_id': flow_id, 'Time': t, 'Length': r_len, 'Is_malicious': is_malicious})

    out = pd.DataFrame(rows)
    if out.empty:
        return out

    out = out.sort_values(['flow_id','Time']).reset_index(drop=True)

    # window index (groups of 10 packets within each flow)
    out['win'] = out.groupby('flow_id').cumcount() // 10

    # unique integer series id per (flow_id, win) WITHOUT bit operations
    out['id'] = pd.factorize(list(zip(out['flow_id'].astype('int64'), out['win'].astype('int64'))))[0].astype('int64')

    return out

In [6]:
def run_process(a,b,x):
    from tsfresh import extract_features, select_features
    from tsfresh.utilities.dataframe_functions import impute
    from tsfresh import extract_features
    from tsfresh.feature_selection.relevance import calculate_relevance_table

    df_malicious = a.copy()
    df_benign    = b.copy()

    for df in (df_malicious, df_benign):
        if 'id' not in df.columns:
            df.reset_index(drop=True, inplace=True)
            df['id'] = (df.index // 10).astype(int)

    tf1=tsfresh.extract_features(df_malicious,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf1['class']= 1

    
    
    tf2=tsfresh.extract_features(df_benign,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf2['class']= 0


    tf2.columns = tf1.columns

    features=pd.concat([tf1,tf2])


    features2 = features.copy()
    features2.reset_index(drop=True, inplace=True)
    
    y = pd.Series(data = features2['class'], index=features2.index)
    
    from tsfresh.examples import load_robot_execution_failures
    from tsfresh import extract_features, select_features
    from tsfresh.feature_selection.relevance import calculate_relevance_table

    relevance_table = calculate_relevance_table(features2, y)
    relevance_table = relevance_table[relevance_table.relevant]
    relevance_table.sort_values("p_value", inplace=True)

    relevance_table
    
    best_features = relevance_table[relevance_table['p_value'] <= 0.05]

    df_ML = pd.DataFrame()

    for pkt in best_features:
        df_ML[best_features.feature] = features[best_features.feature]

    final = ML_Process(df_ML,x)

    return final

In [7]:
def ML_Process(df_ML,x):
    df_results = x.copy() 
    print('let the ml starts')
  
    from sklearn import neighbors, metrics
    from sklearn.preprocessing import LabelEncoder

    #X = df_finalized[['Time', 'Length','Protocol']].values
    X = df_ML.drop('class',axis=1).to_numpy()
    #y = df_finalized[['Is_malicious']]
    y = df_ML['class'].to_numpy()


    #print(X,y)
    
    from sklearn.model_selection import train_test_split
    Le = LabelEncoder()
    for i in range(len(X[0])):
        X[:, i] = Le.fit_transform(X[:, i])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8675309)

    
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.naive_bayes import GaussianNB
    #from xgboost import XGBClassifier
    from sklearn import model_selection
    from sklearn.utils import class_weight
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    import numpy as np
    import pandas as pd
    y_train = y_train.ravel()
    dfs = []
    models = [
          ('LogReg', LogisticRegression()), 
          #('RF', RandomForestClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC()), 
          ('GNB', GaussianNB())
          #('XGB', XGBClassifier())
            ]
    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
    target_names = ['malignant', 'benign']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, 
                                                    scoring=scoring)

        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
        df_resulta = pd.concat([df_results] + dfs, ignore_index=True)
        final = pd.concat(dfs, ignore_index=True)
        print(final)

    return(final)


In [8]:
def get_n_groups(df: pd.DataFrame, group_col: str, n_groups: int, random_state: int = 42) -> pd.DataFrame:
    """
    Randomly pick n_groups distinct values of `group_col` and return all rows that belong to them.
    Useful when your units are flows or 10-packet windows (e.g., group_col='id' or 'flow_id').
    """
    uniq = df[group_col].drop_duplicates()
    n_pick = min(int(n_groups), len(uniq))
    chosen = uniq.sample(n=n_pick, random_state=random_state)
    return df[df[group_col].isin(chosen)].reset_index(drop=True)

In [9]:
# filter the benign data
df_benign_flows = df_w[df_w['NST_B_Label'] == 0].copy()
print("Benign flows:", len(df_benign_flows))

Benign flows: 36648


In [10]:
df_benign_packets = flows_to_pseudopackets(df_benign_flows, is_malicious=0)
print(df_benign_packets.head())

   flow_id          Time  Length  Is_malicious  win  id
0        0  1.678441e+09    59.0             0    0   0
1        0  1.678441e+09    59.0             0    0   0
2        0  1.678441e+09    59.0             0    0   0
3        0  1.678441e+09    59.0             0    0   0
4        0  1.678441e+09    59.0             0    0   0


In [11]:
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=1200)
len(df_benign)

10260

In [12]:
# Scenario 2; Throttles
    
    # 1) THR: %10 (Stealthy)

df_malicious = pd.concat([df33])
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=1150)

# if df_malicious lacks a per-10-packet window id, add one:
if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  # OK if malicious packets are contiguous

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_THR_10_s2 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)

malicious: 9880
benign: 9835
0 NAN in malicious!
0 NAN in benign!
After droppping NAN rows: 
malicious: 9880
benign: 9835


Feature Extraction:   0%|          | 0/198 [00:00<?, ?it/s]

Feature Extraction:   1%|          | 1/198 [00:00<01:30,  2.18it/s]

Feature Extraction:   1%|          | 2/198 [00:00<00:57,  3.42it/s]

Feature Extraction:   4%|▎         | 7/198 [00:00<00:14, 13.10it/s]

...

Feature Extraction: 100%|██████████| 230/230 [00:03<00:00, 74.37it/s] 

let the ml starts


LogReg
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00       282
      benign       1.00      1.00      1.00       253

    accuracy                           1.00       535
   macro avg       1.00      1.00      1.00       535
weighted avg       1.00      1.00      1.00       535

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  0.410686    0.008306            1.0                      1.0   
1  0.009902    0.007661            1.0                      1.0   
2  0.009645    0.007373            1.0                      1.0   
3  0.009881    0.007348            1.0                      1.0   
4  0.010161    0.007366            1.0                      1.0   

   test_recall_weighted  test_f1_weighted  test_roc_auc   model  
0                   1.0               1.0           1.0  LogReg  
1                   1.0               1.0           1.0  LogReg  
2                   1.0               1.0           1.0  LogReg  

KNN
              precision    recall  f1-score   support

   malignant       0.99      1.00      0.99       282
      benign       1.00      0.99      0.99       253

    accuracy                           0.99       535
   macro avg       0.99      0.99      0.99       535
weighted avg       0.99      0.99      0.99       535

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  0.410686    0.008306       1.000000                  1.00000   
1  0.009902    0.007661       1.000000                  1.00000   
2  0.009645    0.007373       1.000000                  1.00000   
3  0.009881    0.007348       1.000000                  1.00000   
4  0.010161    0.007366       1.000000                  1.00000   
5  0.001009    0.369631       1.000000                  1.00000   
6  0.001009    0.152335       0.990654                  0.99082   
7  0.001761    0.160988       1.000000                  1.00000   
8  0.001178    0.103589       1.000000                  1.00000   

SVM
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00       282
      benign       1.00      1.00      1.00       253

    accuracy                           1.00       535
   macro avg       1.00      1.00      1.00       535
weighted avg       1.00      1.00      1.00       535

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0   0.410686    0.008306       1.000000                  1.00000   
1   0.009902    0.007661       1.000000                  1.00000   
2   0.009645    0.007373       1.000000                  1.00000   
3   0.009881    0.007348       1.000000                  1.00000   
4   0.010161    0.007366       1.000000                  1.00000   
5   0.001009    0.369631       1.000000                  1.00000   
6   0.001009    0.152335       0.990654                  0.99082   
7   0.001761    0.160988       1.000000                  1.00000   
8   0.001178    0.103589       1.000000                  

In [13]:
# 2) THR: %50 (Robust)

df_malicious = pd.concat([df3,df34])
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=1200)

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_THR_50_s2 = run_process(df_malicious,df_benign,df_results)

end = timer()
print(end - start)

malicious: 9925
benign: 10260
0 NAN in malicious!
0 NAN in benign!
After droppping NAN rows: 
malicious: 9925
benign: 10260


Feature Extraction:   0%|          | 0/199 [00:00<?, ?it/s]

Feature Extraction:   1%|          | 1/199 [00:00<01:30,  2.19it/s]

Feature Extraction:   3%|▎         | 5/199 [00:00<00:18, 10.73it/s]

Feature Extraction:   5%|▌         | 10/199 [00:00<00:09, 20.21it/s]

...

Feature Extraction: 100%|██████████| 240/240 [00:03<00:00, 108.88it/s]

Feature Extraction: 100%|██████████| 240/240 [00:03<00:00, 71.56it/s] 

let the ml starts
LogReg


              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00       304
      benign       1.00      1.00      1.00       245

    accuracy                           1.00       549
   macro avg       1.00      1.00      1.00       549
weighted avg       1.00      1.00      1.00       549

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  0.013798    0.007869       0.996960                 0.996979   
1  0.014132    0.007869       1.000000                 1.000000   
2  0.012941    0.022782       1.000000                 1.000000   
3  0.028260    0.007429       1.000000                 1.000000   
4  0.017390    0.007579       0.996951                 0.996970   

   test_recall_weighted  test_f1_weighted  test_roc_auc   model  
0              0.996960          0.996961           1.0  LogReg  
1              1.000000          1.000000           1.0  LogReg  
2              1.000000          1.000000           1.0  LogReg  
3     

KNN
              precision    recall  f1-score   support

   malignant       0.99      1.00      1.00       304
      benign       1.00      0.99      1.00       245

    accuracy                           1.00       549
   macro avg       1.00      1.00      1.00       549
weighted avg       1.00      1.00      1.00       549

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  0.013798    0.007869       0.996960                 0.996979   
1  0.014132    0.007869       1.000000                 1.000000   
2  0.012941    0.022782       1.000000                 1.000000   
3  0.028260    0.007429       1.000000                 1.000000   
4  0.017390    0.007579       0.996951                 0.996970   
5  0.000945    0.312652       0.993921                 0.993921   
6  0.001096    0.217309       1.000000                 1.000000   
7  0.001113    0.175367       1.000000                 1.000000   
8  0.001200    0.185921       1.000000                 1.000000   

In [14]:
 # 3) THR: %100 (Aggressive)
    
df_malicious = pd.concat([df1,df2,df4,df5,df6,df7,df35])
df_benign    = df_benign_packets

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_THR_100_s2 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)


malicious: 1520681
benign: 1276276
0 NAN in malicious!
0 NAN in benign!


After droppping NAN rows: 
malicious: 1520681
benign: 1276276


Feature Extraction:   0%|          | 0/240 [00:00<?, ?it/s]

Feature Extraction:   0%|          | 1/240 [00:57<3:50:47, 57.94s/it]

Feature Extraction:   1%|          | 2/240 [00:58<1:36:15, 24.27s/it]

...

let the ml starts


LogReg
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36566
      benign       1.00      1.00      1.00     38152

    accuracy                           1.00     74718
   macro avg       1.00      1.00      1.00     74718
weighted avg       1.00      1.00      1.00     74718

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0  12.242797    0.065123       0.999933                 0.999933   
1  10.789727    0.042655       1.000000                 1.000000   
2  10.468168    0.064383       1.000000                 1.000000   
3  10.398138    0.076342       1.000000                 1.000000   
4  10.292497    0.078456       1.000000                 1.000000   

   test_recall_weighted  test_f1_weighted  test_roc_auc   model  
0              0.999933          0.999933      0.999997  LogReg  
1              1.000000          1.000000      1.000000  LogReg  
2              1.000000          1.000000      1.000000  Lo

KNN
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36566
      benign       1.00      1.00      1.00     38152

    accuracy                           1.00     74718
   macro avg       1.00      1.00      1.00     74718
weighted avg       1.00      1.00      1.00     74718

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0  12.242797    0.065123       0.999933                 0.999933   
1  10.789727    0.042655       1.000000                 1.000000   
2  10.468168    0.064383       1.000000                 1.000000   
3  10.398138    0.076342       1.000000                 1.000000   
4  10.292497    0.078456       1.000000                 1.000000   
5   0.195042   14.358636       0.999777                 0.999777   
6   0.196544   14.338774       0.999799                 0.999799   
7   0.213248   14.555155       0.999911                 0.999911   
8   0.194191   14.565329       0.999888                 0

SVM
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36566
      benign       1.00      1.00      1.00     38152

    accuracy                           1.00     74718
   macro avg       1.00      1.00      1.00     74718
weighted avg       1.00      1.00      1.00     74718

     fit_time  score_time  test_accuracy  test_precision_weighted  \
0   12.242797    0.065123       0.999933                 0.999933   
1   10.789727    0.042655       1.000000                 1.000000   
2   10.468168    0.064383       1.000000                 1.000000   
3   10.398138    0.076342       1.000000                 1.000000   
4   10.292497    0.078456       1.000000                 1.000000   
5    0.195042   14.358636       0.999777                 0.999777   
6    0.196544   14.338774       0.999799                 0.999799   
7    0.213248   14.555155       0.999911                 0.999911   
8    0.194191   14.565329       0.999888        

GNB
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36566
      benign       1.00      1.00      1.00     38152

    accuracy                           1.00     74718
   macro avg       1.00      1.00      1.00     74718
weighted avg       1.00      1.00      1.00     74718

     fit_time  score_time  test_accuracy  test_precision_weighted  \
0   12.242797    0.065123       0.999933                 0.999933   
1   10.789727    0.042655       1.000000                 1.000000   
2   10.468168    0.064383       1.000000                 1.000000   
3   10.398138    0.076342       1.000000                 1.000000   
4   10.292497    0.078456       1.000000                 1.000000   
5    0.195042   14.358636       0.999777                 0.999777   
6    0.196544   14.338774       0.999799                 0.999799   
7    0.213248   14.555155       0.999911                 0.999911   
8    0.194191   14.565329       0.999888        

In [15]:
## S3; In-browser VS Binary

#1) In-Browser
df_malicious = pd.concat([df3,df5,df6,df7,df32,df33,df34,df35])
df_benign = get_n_groups(df_benign_packets, group_col='id', n_groups=35200)

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_In_s3 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)

malicious: 305874
benign: 305555
0 NAN in malicious!
0 NAN in benign!
After droppping NAN rows: 
malicious: 305874
benign: 305555


Feature Extraction:   0%|          | 0/239 [00:00<?, ?it/s]

Feature Extraction:   0%|          | 1/239 [00:11<46:27, 11.71s/it]

Feature Extraction:   1%|          | 2/239 [00:11<19:39,  4.98s/it]

Feature Extraction:   1%|▏         | 3/239 [00:12<11:04,  2.81s/it]

...

let the ml starts


LogReg
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00      8792
      benign       1.00      1.00      1.00      7655

    accuracy                           1.00     16447
   macro avg       1.00      1.00      1.00     16447
weighted avg       1.00      1.00      1.00     16447

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  6.037898    0.050142       0.999797                 0.999797   
1  5.196273    0.050065       0.999899                 0.999899   
2  9.335840    0.034183       0.999797                 0.999797   
3  5.620776    0.041847       1.000000                 1.000000   
4  5.092360    0.054731       0.999696                 0.999696   

   test_recall_weighted  test_f1_weighted  test_roc_auc   model  
0              0.999797          0.999797      0.999969  LogReg  
1              0.999899          0.999899      1.000000  LogReg  
2              0.999797          0.999797      0.999998  LogReg  

KNN
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00      8792
      benign       1.00      1.00      1.00      7655

    accuracy                           1.00     16447
   macro avg       1.00      1.00      1.00     16447
weighted avg       1.00      1.00      1.00     16447

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  6.037898    0.050142       0.999797                 0.999797   
1  5.196273    0.050065       0.999899                 0.999899   
2  9.335840    0.034183       0.999797                 0.999797   
3  5.620776    0.041847       1.000000                 1.000000   
4  5.092360    0.054731       0.999696                 0.999696   
5  0.038536    0.840205       0.998885                 0.998886   
6  0.037294    0.717844       0.999595                 0.999595   
7  0.037427    0.715234       0.999189                 0.999190   
8  0.037458    0.717491       0.999493                 0.999493   

SVM
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00      8792
      benign       1.00      1.00      1.00      7655

    accuracy                           1.00     16447
   macro avg       1.00      1.00      1.00     16447
weighted avg       1.00      1.00      1.00     16447

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0   6.037898    0.050142       0.999797                 0.999797   
1   5.196273    0.050065       0.999899                 0.999899   
2   9.335840    0.034183       0.999797                 0.999797   
3   5.620776    0.041847       1.000000                 1.000000   
4   5.092360    0.054731       0.999696                 0.999696   
5   0.038536    0.840205       0.998885                 0.998886   
6   0.037294    0.717844       0.999595                 0.999595   
7   0.037427    0.715234       0.999189                 0.999190   
8   0.037458    0.717491       0.999493                 0

GNB
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00      8792
      benign       1.00      1.00      1.00      7655

    accuracy                           1.00     16447
   macro avg       1.00      1.00      1.00     16447
weighted avg       1.00      1.00      1.00     16447

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0   6.037898    0.050142       0.999797                 0.999797   
1   5.196273    0.050065       0.999899                 0.999899   
2   9.335840    0.034183       0.999797                 0.999797   
3   5.620776    0.041847       1.000000                 1.000000   
4   5.092360    0.054731       0.999696                 0.999696   
5   0.038536    0.840205       0.998885                 0.998886   
6   0.037294    0.717844       0.999595                 0.999595   
7   0.037427    0.715234       0.999189                 0.999190   
8   0.037458    0.717491       0.999493                 0

In [16]:
#2) Binary
df_malicious = pd.concat([df1,df2,df4])
df_benign    = df_benign_packets

if 'id' not in df_malicious.columns:
    df_malicious = df_malicious.sort_values('Time').reset_index(drop=True)
    df_malicious['id'] = (df_malicious.index // 10).astype('int64')  

print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

print("{} NAN in malicious!".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign!".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

print("After droppping NAN rows: ")
print("malicious: {}".format(len(df_malicious)))
print("benign: {}".format(len(df_benign)))

start = timer()

results_Host_s3 = run_process(df_malicious,df_benign,df_results)


end = timer()
print(end - start)

malicious: 1251356
benign: 1276276
0 NAN in malicious!
0 NAN in benign!


After droppping NAN rows: 
malicious: 1251356
benign: 1276276


Feature Extraction:   0%|          | 0/240 [00:00<?, ?it/s]

Feature Extraction:   0%|          | 1/240 [00:46<3:06:03, 46.71s/it]

Feature Extraction:   1%|          | 2/240 [00:47<1:17:51, 19.63s/it]

...

let the ml starts


LogReg
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36802
      benign       1.00      1.00      1.00     31183

    accuracy                           1.00     67985
   macro avg       1.00      1.00      1.00     67985
weighted avg       1.00      1.00      1.00     67985

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  6.875749    0.061086       0.999975                 0.999975   
1  6.747869    0.098339       1.000000                 1.000000   
2  5.612452    0.066159       0.999975                 0.999975   
3  6.626446    0.068129       1.000000                 1.000000   
4  6.827224    0.106039       1.000000                 1.000000   

   test_recall_weighted  test_f1_weighted  test_roc_auc   model  
0              0.999975          0.999975           1.0  LogReg  
1              1.000000          1.000000           1.0  LogReg  
2              0.999975          0.999975           1.0  LogReg  

KNN
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36802
      benign       1.00      1.00      1.00     31183

    accuracy                           1.00     67985
   macro avg       1.00      1.00      1.00     67985
weighted avg       1.00      1.00      1.00     67985

   fit_time  score_time  test_accuracy  test_precision_weighted  \
0  6.875749    0.061086       0.999975                 0.999975   
1  6.747869    0.098339       1.000000                 1.000000   
2  5.612452    0.066159       0.999975                 0.999975   
3  6.626446    0.068129       1.000000                 1.000000   
4  6.827224    0.106039       1.000000                 1.000000   
5  0.185882   11.733971       0.999828                 0.999828   
6  0.177389   11.629555       0.999926                 0.999926   
7  0.178706   11.673275       0.999877                 0.999877   
8  0.180504   11.686674       0.999902                 0.999902   

SVM
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36802
      benign       1.00      1.00      1.00     31183

    accuracy                           1.00     67985
   macro avg       1.00      1.00      1.00     67985
weighted avg       1.00      1.00      1.00     67985

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0   6.875749    0.061086       0.999975                 0.999975   
1   6.747869    0.098339       1.000000                 1.000000   
2   5.612452    0.066159       0.999975                 0.999975   
3   6.626446    0.068129       1.000000                 1.000000   
4   6.827224    0.106039       1.000000                 1.000000   
5   0.185882   11.733971       0.999828                 0.999828   
6   0.177389   11.629555       0.999926                 0.999926   
7   0.178706   11.673275       0.999877                 0.999877   
8   0.180504   11.686674       0.999902                 0

GNB
              precision    recall  f1-score   support

   malignant       1.00      1.00      1.00     36802
      benign       1.00      1.00      1.00     31183

    accuracy                           1.00     67985
   macro avg       1.00      1.00      1.00     67985
weighted avg       1.00      1.00      1.00     67985

    fit_time  score_time  test_accuracy  test_precision_weighted  \
0   6.875749    0.061086       0.999975                 0.999975   
1   6.747869    0.098339       1.000000                 1.000000   
2   5.612452    0.066159       0.999975                 0.999975   
3   6.626446    0.068129       1.000000                 1.000000   
4   6.827224    0.106039       1.000000                 1.000000   
5   0.185882   11.733971       0.999828                 0.999828   
6   0.177389   11.629555       0.999926                 0.999926   
7   0.178706   11.673275       0.999877                 0.999877   
8   0.180504   11.686674       0.999902                 0